## FSDP Tutorial

In this tutorial, we will walk through the implementation of Fully Sharded Data Parallel (FSDP) with Zero2 sharding strategy in `thunder`.

#### Introduction

In recent times, the LLM models have grown so large that all the model parameters don't fit on a single GPU. To circumvent this problem, there are various strategies like Tensor Parallel, Pipeline Parallel, Fully Sharded Data Parallel, etc to train these large models. In this tutorial, we discuss and implement Zero2 strategy for Fully Sharded Data Parallel (FSDP).


#### What is Zero2 strategy for FSDP?

In this strategy, we shard the model parameters across all the availabe GPUs. That is each GPU holds onto only a chunk of the parameter. During the forward pass, all GPUs call `all_gather` communication primitive to gather the parameters from other GPUs. Unlike Zero3 strategy which frees the parameter after forward pass, we save these unsharded parameters for backward pass. This is to save the overhead of extra communication. In the backward pass, we utilize the saved parameters and compute the gradients. Once the gradients are computed, we use `reduce_scatter` communication primitive to reduce (average) the gradients across all GPUs and scatter those gradients so that a given GPU holds only a chunk of gradient.

For more information on FSDP, we recommend reading

1. PyTorch FSDP: Experiences on Scaling Fully Sharded Data Parallel - [Link](https://arxiv.org/abs/2304.11277)
2. ZeRO: Memory Optimizations Toward Training Trillion Parameter Models - [Link](https://arxiv.org/abs/1910.02054)

#### Example Model

For this example we will have a simple model `Linear(Tanh(Linear(x)))` which will be sharded over 2 GPUs

**NOTE**: We are generating the abstract trace so we don't actually need a system with 2 GPUs for this. It is only required when we execute this trace.

In [1]:
import torch
import torch.distributed
import thunder
import thunder.distributed
from IPython.display import Code

In [2]:
device='cuda'
dim = 64
def create_model():
    layers = [torch.nn.Linear(dim, dim, bias=False),
              torch.nn.Tanh(),
              torch.nn.Linear(dim, dim, bias=False)]
    return torch.nn.Sequential(*layers).to(device)

# Model
model = create_model()
# Input
x = torch.randn(dim, dim, device=device)


# we want to obtain a functional version of our model. The JIT does that internally and we reach into those
# internals here
thunder_model = thunder.jit(model)
cache_rec, i_, _ = thunder.compile_data(thunder_model).get_computation_and_inputs(x)
computation_trace = cache_rec.computation_traces[0]


In [3]:
def wrap_as_highlighted_code(trace):
    return Code(str(trace), language="python")

We can show the functional version:

In [4]:
wrap_as_highlighted_code(computation_trace)

# Constructed by Dead Code Elimination (took 0 milliseconds)
import torch
import torch.nn.functional
from thunder.executors.torchex import no_autocast

@torch.no_grad()
@no_autocast()
def augmented_forward_fn(*args):
  # args: "Collection" 
  t0, \
  t1, \
  t2, \
  = args
  t3 = torch.nn.functional.linear(t0, t1, None)  # t3: "cuda:0 f32[64, 64]"
    # t3 = ltorch.linear(t0, t1, None)  # t3: "cuda:0 f32[64, 64]"
      # t3 = prims.linear(t0, t1, None)  # t3: "cuda:0 f32[64, 64]"
  [t4] = nvFusion0(t3)
    # t4 = prims.tanh(t3)  # t4: "cuda:0 f32[64, 64]"
  t5 = torch.nn.functional.linear(t4, t2, None)  # t5: "cuda:0 f32[64, 64]"
    # t5 = ltorch.linear(t4, t2, None)  # t5: "cuda:0 f32[64, 64]"
      # t5 = prims.linear(t4, t2, None)  # t5: "cuda:0 f32[64, 64]"
  return {'output': t5, 'flat_args': [t0, t1, t2], 'flat_output': (t5,)}, ((t0, t2, t4), ())

#### Step 1 : Configuration

For our implementation of FSDP, we will generate the trace where we are sharding our model over 2 GPU

In [5]:
# FSDP Config 
# Usually these values are set in the environment by `torchrun` but for this example
# we will set them ourselves
world_size = 2  # We have two processes.
global_rank = 0  # Current process is the very first process.

#### Step 2: Function to shard parameters

Next step is to write a function which will actually shard the parameters over 0-dim.

In [6]:
# NOTE: We shard over 0th dimension of the param.
def shard_param(param: torch.Tensor, rank: int, world_size: int, name: str) -> None:
    # We will keep it simple and error if param's 0th dim is not divisible by ``world_size``.
    # Alternative is that we can pad our parameters so that they are divisible by `world_size`.
    assert param.shape[0] % world_size == 0,(
        f"Current sharding requires the first dimension of the parameter {name!r} ({param.shape[0]})"
        f" to be divisible by the world size ({world_size})"
    )
    chunk_size = param.shape[0] // world_size

    # rank helps us determine which chunk of the parameter we will hold.
    shard = param.data.narrow(0, chunk_size * rank, chunk_size).clone()
    param.data = shard

# Shard each parameter of the model
for param_name, param in model.named_parameters():
    shard_param(param, global_rank, world_size, param_name)
    # Mark the param to denote that it is sharded.
    # This is required by the synchronization primitive we will use below.
    param.ddp_type = thunder.core.proxies.DDPType.FULLY_SHARDED

In [7]:
# Verify our model looks as expected
model

Sequential(
  (0): Linear(in_features=64, out_features=64, bias=False)
  (1): Tanh()
  (2): Linear(in_features=64, out_features=64, bias=False)
)

In [8]:
# Let us verify that we have actually sharded the parameters.
# Checking if the weight of 1st Linear layer is sharded over 0th dim.
assert model[0].weight.shape == (dim / world_size, dim)

#### Step 3: Add an operation to synchronize the parameters before calling the model.forward.

We have to create a process group. This is needed because the synchronization primitive `synchronize` that we will use to gather and scatter our weights in forward and backward requires a process group.

In [9]:
# Create a process group
options = torch.distributed.distributed_c10d.ProcessGroup.Options(backend="nccl")
process_group = torch.distributed.distributed_c10d.ProcessGroup(torch.distributed.distributed_c10d.Store(),
                                                     global_rank, world_size, options)
torch.distributed.distributed_c10d.GroupMember.WORLD = process_group


Because we are trying to play tricks with the traces and skip the part that inserts the synchronization automatically but also does the translation from PyTorch to thunder, we need to drop one layer of the trace to apply this manually.
(This is really hacky, don't try it at home!)

In [10]:
### DON'T TRY THIS AT HOME
computation_trace.bound_symbols[2].sym = cache_rec.computation_traces[0].bound_symbols[2].subsymbols[0].sym
if cache_rec.computation_traces[0].bound_symbols[3].subsymbols:
    computation_trace.bound_symbols[3] = cache_rec.computation_traces[0].bound_symbols[3].subsymbols[0]
computation_trace.bound_symbols[4].sym = cache_rec.computation_traces[0].bound_symbols[4].subsymbols[0].sym

wrap_as_highlighted_code(computation_trace)

# Constructed by Dead Code Elimination (took 0 milliseconds)
import thunder
import thunder.core.prims as prims
import thunder.torch as ltorch
import torch
import torch.nn.functional
from thunder.executors.torchex import no_autocast

@torch.no_grad()
@no_autocast()
def augmented_forward_fn(*args):
  # args: "Collection" 
  t0, \
  t1, \
  t2, \
  = args
  t3 = ltorch.linear(t0, t1, None)  # t3: "cuda:0 f32[64, 64]"
    # t3 = ltorch.linear(t0, t1, None)  # t3: "cuda:0 f32[64, 64]"
      # t3 = prims.linear(t0, t1, None)  # t3: "cuda:0 f32[64, 64]"
  t4 = prims.tanh(t3)  # t4: "cuda:0 f32[64, 64]"
  t5 = ltorch.linear(t4, t2, None)  # t5: "cuda:0 f32[64, 64]"
    # t5 = ltorch.linear(t4, t2, None)  # t5: "cuda:0 f32[64, 64]"
      # t5 = prims.linear(t4, t2, None)  # t5: "cuda:0 f32[64, 64]"
  return {'output': t5, 'flat_args': [t0, t1, t2], 'flat_output': (t5,)}, ((t0, t2, t4), ())

In [11]:
# now we have a  functional version of the model which
# takes as inputs the expected arguments and all the parameters.
functional_forward = computation_trace.python_callable()

# This function creates a model with synchronization
# before calling the forward pass.
def model_with_syncs(x, *params):
    # We call `prims.synchronize` on all the parameters.
    # This is essentially calling `all_gather` so that we have the complete
    # parameter before we actually to the forward computation.
    unsharded_params = []
    for param in params:
        unsharded_params.append(thunder.distributed.prims.synchronize(param, process_group))

    return functional_forward(x, *unsharded_params)

Let us now see what the trace of our model looks like with all the synchronization.

Two main observations regarding the below trace 
1. We can observe the `prims.synchronize` that we inserted using `model_with_syncs`.
2. Output of the `prims.synchronize` have the shape of unsharded (original) parameter.

With this, we have implemented the FSDP for the forward pass of our model.

In [12]:
trace = thunder.trace()(model_with_syncs, x, *model.parameters())

wrap_as_highlighted_code(trace)

# Constructed by Dead Code Elimination (took 0 milliseconds)
import thunder
import thunder.core.prims as prims
import thunder.distributed.prims
import thunder.torch as ltorch
import torch
from thunder.executors.torchex import no_autocast

@torch.no_grad()
@no_autocast()
def model_with_syncs(x, *params):
  # x: "cuda:0 f32[64, 64]" 
  # params: "Collection" 
  t0, \
  t1, \
  = params
  t2 = thunder.distributed.prims.synchronize(t0, _torch_distributed_distributed_c10d_ProcessGroup_0)  # t2: "cuda:0 f32[64, 64]"
  t3 = thunder.distributed.prims.synchronize(t1, _torch_distributed_distributed_c10d_ProcessGroup_0)  # t3: "cuda:0 f32[64, 64]"
  t4 = ltorch.linear(x, t2, None)  # t4: "cuda:0 f32[64, 64]"
    # t4 = prims.linear(x, t2, None)  # t4: "cuda:0 f32[64, 64]"
  t5 = prims.tanh(t4)  # t5: "cuda:0 f32[64, 64]"
  t6 = ltorch.linear(t5, t3, None)  # t6: "cuda:0 f32[64, 64]"
    # t6 = prims.linear(t5, t3, None)  # t6: "cuda:0 f32[64, 64]"
  return ({'output': t6, 'flat_args': [x, t2, t3], 'flat_output': (t6,)}, ((x, t3, t5), ()))

For backward, we don't have to do anything because `thunder` already knows how to compute the backward of `prims.synchronize`. We can verify that by using the `value_and_grad` transform to generate the complete forward and backward trace together.

Observations for the trace below:
1. `prims.synchronize` from previous trace is now decomposed into `prims.all_gather` and `prims.wait`. So, we can clearly see that we make a communication call to gather the parameter (which is asynchronous) and wait till we have the complete parameter.
2. At the end of the trace (after the forward and the backward computation), we see calls to `prims.reduce_scatter` and `prims.wait`. This takes care of reducing the gradients across all the GPUs and sharding them. One thing to note, for averaging gradients with low dynamic range dtype like `float16`, if we naively sum the gradients across GPUs before dividing by `world_size`, it can lead to overflows. So we scale the gradient tensor with `world_size`, before calling `reduce_scatter` with `sum` reduction to effectively average the gradients without overflow.

In [13]:
from thunder.core.transforms import value_and_grad

forward_and_backward_model = value_and_grad(model_with_syncs)

forward_backward_trace = thunder.trace()(forward_and_backward_model, x, *model.parameters())

wrap_as_highlighted_code(forward_backward_trace)

# Constructed by Dead Code Elimination (took 1 milliseconds)
import thunder
import thunder.core.devices as devices
import thunder.core.dtypes as dtypes
import thunder.core.prims as prims
import thunder.distributed.prims
import thunder.torch as ltorch
import torch
from thunder.executors.torchex import no_autocast

@torch.no_grad()
@no_autocast()
def _value_and_grad(*args):
  # args: "Collection" 
  t0, \
  t1, \
  t2, \
  = args
  t3 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t3: "cuda:0 f32[64, 64]"
  t4 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t4: "cuda:0 f32[64, 64]"
  t5 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t5: "cuda:0 f32[64, 64]"
  t6 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t6: "cuda:0 f32[64, 64]"
  t7 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t7: "cuda:0 f32[64, 64]"
  t8 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t8: "cuda:0 f32[64, 64]"
  t9 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t9: "cuda:0 f32[64, 64]"
  t10 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t10: "cuda:0 f32[64, 64]"
  p11 = thunder.distributed.prims.all_gather(t1, _torch_distributed_distributed_c10d_ProcessGroup_0, True)  # p11: "FUTURE cuda:0 f32[64, 64]"
  t12 = thunder.distributed.prims.wait(p11)  # t12: "cuda:0 f32[64, 64]"
  p13 = thunder.distributed.prims.all_gather(t2, _torch_distributed_distributed_c10d_ProcessGroup_0, True)  # p13: "FUTURE cuda:0 f32[64, 64]"
  t14 = thunder.distributed.prims.wait(p13)  # t14: "cuda:0 f32[64, 64]"
  t15 = prims.linear(t0, t12, None)  # t15: "cuda:0 f32[64, 64]"
  t16 = prims.tanh(t15)  # t16: "cuda:0 f32[64, 64]"
  t17 = prims.linear(t16, t14, None)  # t17: "cuda:0 f32[64, 64]"
  t18 = prims.add(t6, t7)  # t18: "cuda:0 f32[64, 64]"
  t19 = prims.add(t3, t8)  # t19: "cuda:0 f32[64, 64]"
  t20 = prims.add(t5, t9)  # t20: "cuda:0 f32[64, 64]"
  t21 = ltorch.reshape(t18, -1, 64)  # t21: "cuda:0 f32[64, 64]"
    # t21 = prims.reshape(t18, (64, 64))  # t21: "cuda:0 f32[64, 64]"
  t22 = ltorch.matmul(t21, t14)  # t22: "cuda:0 f32[64, 64]"
    # t22 = prims.matmul(t21, t14)  # t22: "cuda:0 f32[64, 64]"
  t23 = ltorch.reshape(t18, -1, 64)  # t23: "cuda:0 f32[64, 64]"
    # t23 = prims.reshape(t18, (64, 64))  # t23: "cuda:0 f32[64, 64]"
  t24 = prims.transpose(t23, (1, 0))  # t24: "cuda:0 f32[64, 64]"
  t25 = ltorch.reshape(t16, -1, 64)  # t25: "cuda:0 f32[64, 64]"
    # t25 = prims.reshape(t16, (64, 64))  # t25: "cuda:0 f32[64, 64]"
  t26 = ltorch.matmul(t24, t25)  # t26: "cuda:0 f32[64, 64]"
    # t26 = prims.matmul(t24, t25)  # t26: "cuda:0 f32[64, 64]"
  t27 = prims.add(t10, t22)  # t27: "cuda:0 f32[64, 64]"
  t28 = prims.add(t20, t26)  # t28: "cuda:0 f32[64, 64]"
  t29 = ltorch.mul(t16, t16)  # t29: "cuda:0 f32[64, 64]"
    # t29 = prims.mul(t16, t16)  # t29: "cuda:0 f32[64, 64]"
  t30 = ltorch.sub(1, t29, alpha=None)  # t30: "cuda:0 f32[64, 64]"
    # _ = prims.convert_element_type(1, float)
    # t30 = prims.sub(1.0, t29)  # t30: "cuda:0 f32[64, 64]"
  t31 = ltorch.mul(t27, t30)  # t31: "cuda:0 f32[64, 64]"
    # t31 = prims.mul(t27, t30)  # t31: "cuda:0 f32[64, 64]"
  t32 = ltorch.reshape(t31, -1, 64)  # t32: "cuda:0 f32[64, 64]"
    # t32 = prims.reshape(t31, (64, 64))  # t32: "cuda:0 f32[64, 64]"
  t33 = ltorch.matmul(t32, t12)  # t33: "cuda:0 f32[64, 64]"
    # t33 = prims.matmul(t32, t12)  # t33: "cuda:0 f32[64, 64]"
  t34 = ltorch.reshape(t31, -1, 64)  # t34: "cuda:0 f32[64, 64]"
    # t34 = prims.reshape(t31, (64, 64))  # t34: "cuda:0 f32[64, 64]"
  t35 = prims.transpose(t34, (1, 0))  # t35: "cuda:0 f32[64, 64]"
  t36 = ltorch.reshape(t0, -1, 64)  # t36: "cuda:0 f32[64, 64]"
    # t36 = prims.reshape(t0, (64, 64))  # t36: "cuda:0 f32[64, 64]"
  t37 = ltorch.matmul(t35, t36

The above trace, only contains primitive which specifies the semantic of an operation abstractly but doesn't perform the actual computation.

Now we will generate the execution trace which can actually perform the compute.

In the execution trace generated below, we can see that all the primitives have been replaced with actually PyTorch operations. Also, our synchronization primitives have been replaced with PyTorch implementation provided by thunder i.e. `torch_all_gather_prim_impl`, `torch_reduce_scatter_prim_impl`, `torch_wait_prim_impl`.

In [14]:
optimized_trace = thunder.transform_for_execution(forward_backward_trace, executors_list=thunder.get_always_executors())

# Grab the final trace
exec_trace = optimized_trace[-1]
wrap_as_highlighted_code(exec_trace)

# Constructed by Delete Last Used (took 0 milliseconds)
import torch
import torch.nn.functional
from thunder.executors.torchex import no_autocast

@torch.no_grad()
@no_autocast()
def _value_and_grad(*args):
  # args: "Collection" 
  t0, \
  t1, \
  t2, \
  = args
  del args
  t3 = torch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t3: "cuda:0 f32[64, 64]"
    # t3 = ltorch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t3: "cuda:0 f32[64, 64]"
      # t3 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t3: "cuda:0 f32[64, 64]"
  t4 = torch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t4: "cuda:0 f32[64, 64]"
    # t4 = ltorch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t4: "cuda:0 f32[64, 64]"
      # t4 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t4: "cuda:0 f32[64, 64]"
  t5 = torch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t5: "cuda:0 f32[64, 64]"
    # t5 = ltorch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t5: "cuda:0 f32[64, 64]"
      # t5 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t5: "cuda:0 f32[64, 64]"
  t6 = torch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t6: "cuda:0 f32[64, 64]"
    # t6 = ltorch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t6: "cuda:0 f32[64, 64]"
      # t6 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t6: "cuda:0 f32[64, 64]"
  t7 = torch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t7: "cuda:0 f32[64, 64]"
    # t7 = ltorch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t7: "cuda:0 f32[64, 64]"
      # t7 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t7: "cuda:0 f32[64, 64]"
  t8 = torch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t8: "cuda:0 f32[64, 64]"
    # t8 = ltorch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t8: "cuda:0 f32[64, 64]"
      # t8 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t8: "cuda:0 f32[64, 64]"
  t9 = torch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t9: "cuda:0 f32[64, 64]"
    # t9 = ltorch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t9: "cuda:0 f32[64, 64]"
      # t9 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t9: "cuda:0 f32[64, 64]"
  t10 = torch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t10: "cuda:0 f32[64, 64]"
    # t10 = ltorch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t10: "cuda:0 f32[64, 64]"
      # t10 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t10: "cuda:0 f32[64, 64]"
  p11 = torch_all_gather_prim_impl(t1, _torch_distributed_distributed_c10d_ProcessGroup_2, True)  # p11: "FUTURE cuda:0 f32[64, 64]"
  del t1
  t12 = torch_wait_prim_impl(p11)  # t12: "cuda:0 f32[64, 64]"
  del p11
  p13 = torch_all_gather_prim_impl(t2, _torch_distributed_distributed_c10d_ProcessGroup_2, True)  # p13: "FUTURE cuda:0 f32[64, 64]"
  del t2
  t14 = torch_wait_prim_impl(p13)  # t14: "cuda:0 f32[64, 64]"
  del p13
  t15 = torch.nn.functional.linear(t0, t12, None)  # t15: "cuda:0 f32[64, 64]"
    # t15 = ltorch.linear(t0, t12, None)  # t15: "cuda:0 f32[64, 64]"
      # t15 = prims.linear(t0, t12, None)  # t15: "cuda:0 f32[64, 64]"
  t16 = torch.tanh(t15)  # t16: "cuda:0 f32[64, 64]"
    # t16 = ltorch.tanh(t15)  # t16: "cuda:0 f32[64, 64]"
      # t16 = prims.tanh(t15)  # t16: "cuda:0 f32[64, 64]"
  del t15
  t17 = torch.nn.functional.linear(t16, t14, None)  # t17: "cuda:0 f32[64, 64]"
    # t17 = ltorch.linear(t16, t1

#### Step 4 : Running the actual computation

Running the actual computation will require setting up 2 processes and running our above code in both those processes (which can be tricky with Jupyter Notebook). Instead, we will write a small script and run it with `torchrun` which takes care of setting up the processes and relevant state.

**NOTE**: This requires device running this notebook to have at least 2-GPUs

In the example below, we will use `thunder.distributed.fsdp` which does the same as what we did above (with some extra checks). The code below should look familiar as it is roughly all the above pieces in a single script. 

In [15]:
%%writefile thunder_fsdp_simple_example.py

# imports
from thunder.tests.lit_gpt_model import GPT, Config
import torch
import torch.distributed
import thunder
import thunder.distributed
import os

# # # # # # # #
# Create Model
# # # # # # # #

# NOTE: We create the model on CPU.
device='cpu'
dim = 64
def create_model():
    layers = []
    layers.append(torch.nn.Linear(dim, dim))
    layers.append(torch.nn.ReLU())
    layers.append(torch.nn.Linear(dim, dim))
    return torch.nn.Sequential(*layers).to(device)

# Model
model = create_model()
# Input
x = torch.randn(dim, dim, device=device)

# # # # # # # #
# Setup for distributed
# # # # # # # #
torch.distributed.init_process_group(backend='nccl')

rank = int(os.environ["LOCAL_RANK"])

device = f"cuda:{rank}"

# # # # # # # #
# Move inputs to correct device
# # # # # # # #
x = x.to(device)

# # # # # # # #
# Wrap the model in thunder.distributed.fsdp
# # # # # # # #

# thunder.distributed.fsdp takes care of moving the parameter
# shard to the correct GPU for the current process.
cmodel = thunder.jit(thunder.distributed.fsdp(model))

# Run the forward pass.
cmodel(x)

# # # # # # # #
# Check the traces
# # # # # # # #
fwd_traces, bwd_traces = thunder.last_traces(cmodel)

# # # # # # # #
# Print and check to see if they match ours
# # # # # # # #
if rank == 0:
    print(fwd_traces[-1])
    print("*******"* 8)
    print(bwd_traces[-1])

Overwriting thunder_fsdp_simple_example.py


Let us run the above script and check what the trace looks like.

We can observe that forward trace has `torch_all_gather_prim_impl` to gather the parameter before forward pass and the backward trace has `torch_reduce_scatter_prim_impl` to reduce and scatter the gradients back to different GPUs. This is similar to our implementation above.

In [16]:
!torchrun --nproc_per_node=2 thunder_fsdp_simple_example.py

W0314 08:26:39.130000 140292199276608 torch/distributed/run.py:757] 
W0314 08:26:39.130000 140292199276608 torch/distributed/run.py:757] *****************************************
W0314 08:26:39.130000 140292199276608 torch/distributed/run.py:757] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0314 08:26:39.130000 140292199276608 torch/distributed/run.py:757] *****************************************
# Constructed by Delete Last Used (took 0 milliseconds)
import torch
import torch.nn.functional
from thunder.executors.torchex import no_autocast

@torch.no_grad()
@no_autocast()
def augmented_forward_fn(input, t_0_bias, t_2_bias, t_0_weight, t_2_weight):
  # input: "cuda:0 f32[64, 64]" 
  # t_0_bias: "cuda:0 f32[32]" 
  p0 = torch_all_gather_prim_impl(t_0_bias, _torch_distributed_distributed_c10d_ProcessGroup_1, True)  # p0: "FU

#### Conclusion

We have created our implementation of FSDP to shard our model across multiple GPUs. In the process, we also learned that:

1. `thunder` provides us with primitives for synchronization across mutiple GPUs.
2. `thunder` also takes care of implementing the backward support for the synchronization primitives, so we don't have to explicitly do anything to get the backward working.
3. We can just easily apply `thunder.distributed.fsdp` to our model and it will take care of sharding the parameters and also adding synchronizations to our model. Also, we can easily check the modifications by inspecting the traces.